In [ ]:
import pandas as pd
from ortools.linear_solver import pywraplp as OR

In [ ]:
foods = pd.read_csv('diet_neos/neos_foods.csv', index_col=0)
nutrients = pd.read_csv('diet_neos/neos_nutrients.csv', index_col=0)

In [ ]:
def diet(foods, nutrients, integer=False):
    """A model for solving the diet problem.
    
    Args:
        foods (pd.DataFrame): Foods with cost per serving, min and max servings, and nutrients per serving.
        nutrients (pd.DataFrame): Nutrients with min and max bounds.
    """
    FOODS = list(foods.index)                                 # foods
    NUTRIENTS = list(nutrients.index)                         # nutrients
    c = foods['Cost'].to_dict()                               # cost per serving of food 
    f_min = foods['Min'].to_dict()                            # lower bound of food serving
    f_max = foods['Max'].to_dict()                            # upper bound of food serving
    n_min = nutrients['Min'].to_dict()                        # lower bound of nutrient
    n_max = nutrients['Max'].to_dict()                        # upper bound of nutrient  
    a = foods[list(nutrients.index)].transpose().to_dict()    # amt of nutrients per serving of food
    
    # define model
    m = OR.Solver('diet', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
        
    # decision variables
    x = {}    
    for i in FOODS:
        if integer:
            x[i] = m.IntVar(0, m.infinity(), 'x_%s' % (i)) 
        else:
            x[i] = m.NumVar(0, m.infinity(), 'x_%s' % (i)) 
        
    # objective function.
    m.Minimize(sum(c[i]*x[i] for i in FOODS))
    
    # enforce lower and upper bound on food servings
    for i in FOODS:
        m.Add(x[i] >= f_min[i], name='lb_%s' % (i))
        m.Add(x[i] <= f_max[i], name='ub_%s' % (i))
    
    # enforce lower and upper bound on nutrients 
    for j in NUTRIENTS:
        m.Add(sum(a[i][j]*x[i] for i in FOODS) >= n_min[j], name='lb_%s' % (j))
        m.Add(sum(a[i][j]*x[i] for i in FOODS) <= n_max[j], name='ub_%s' % (j))
        
    return m, x

In [ ]:
def solve(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())
    for var in m.variables():
        print(var.name(), ':',  var.solution_value())

In [ ]:
m, x = diet(foods, nutrients)
solve(m)

In [ ]:
m, x = diet(foods, nutrients, integer=True)
solve(m)